# Расчет метрик юнит-экономики с сегментацией когорт

## Анализ датасета

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Case for Homework for students - Data.csv')
df

,data,cohort,uuid,buyer,new buyer,aov,cogs,ltc,month,n_payment
0,5/1/2024,2024-05,be239004-bc93-4732-8d2e-db1c9abc70f9,-,False,0,0.0,0,2024-5,NaN
1,5/1/2024,2024-05,ff709093-05bd-4c07-8d47-20a1e22450df,-,False,0,0.0,8,2024-5,NaN
2,5/1/2024,2024-05,024c15c8-33a9-4f4c-b471-c47a2364dc30,-,False,0,0.0,8,2024-5,NaN
3,5/1/2024,2024-05,3186905a-cf69-40fe-a615-51342cc6158b,-,False,0,0.0,8,2024-5,NaN
4,5/1/2024,2024-05,919338ce-b051-4c7a-aa3c-e7c951644a77,-,False,0,0.0,0,2024-5,NaN
...,...,...,...,...,...,...,...,...,...,...
59313,7/31/2024,2024-07,bff9f722-83d9-4907-ac23-cb76399912c5,-,False,0,0.0,8,2024-7,NaN
59314,7/31/2024,2024-07,c5e8d0e7-2128-4113-a29c-866f2abf1af0,-,False,0,0.0,0,2024-7,NaN
59315,7/31/2024,2024-07,d613f8a5-f0f9-4b7b-bc54-27c62bd03eef,-,False,0,0.0,0,2024-7,NaN
59316,7/31/2024,2024-07,6b1105f3-8759-4475-adc8-07447b6a994f,-,False,0,0.0,0,2024-7,NaN


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59318 entries, 0 to 59317
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   data       59318 non-null  object 
 1   cohort     59318 non-null  object 
 2   uuid       59318 non-null  object 
 3   buyer      59318 non-null  object 
 4   new buyer  59318 non-null  bool   
 5   aov        59318 non-null  int64  
 6   cogs       59318 non-null  float64
 7   ltc        59318 non-null  int64  
 8   month      59318 non-null  object 
 9   n_payment  0 non-null      float64
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 4.1+ MB


In [78]:
df.tail()

,data,cohort,uuid,buyer,new buyer,aov,cogs,ltc,month,n_payment
59313,7/31/2024,2024-07,bff9f722-83d9-4907-ac23-cb76399912c5,-,False,0,0.0,8,2024-7,NaN
59314,7/31/2024,2024-07,c5e8d0e7-2128-4113-a29c-866f2abf1af0,-,False,0,0.0,0,2024-7,NaN
59315,7/31/2024,2024-07,d613f8a5-f0f9-4b7b-bc54-27c62bd03eef,-,False,0,0.0,0,2024-7,NaN
59316,7/31/2024,2024-07,6b1105f3-8759-4475-adc8-07447b6a994f,-,False,0,0.0,0,2024-7,NaN
59317,7/31/2024,2024-07,4324a7b7-2781-4694-92d5-8814d886b8e1,-,False,0,0.0,8,2024-7,NaN


In [3]:
cohorts_df = df.groupby('cohort')['uuid'].nunique()

print(cohorts_df, '\n')

unique_uuids = df['uuid'].nunique()
unique_buyers = df[df['buyer'] != '-']['buyer'].nunique()
count_deals = (df[df['buyer'] != '-'].shape[0])

print("Общее количество уникальных uuid:", unique_uuids, '\n',
      "Общее количество уникальных buyers:", unique_buyers, '\n', 
      "Общее количество покупок:",  count_deals, '\n')

cohort
2024-05     9810
2024-06    14547
2024-07    17094
Name: uuid, dtype: int64 

Общее количество уникальных uuid: 41451 
 Общее количество уникальных buyers: 547 
 Общее количество покупок: 745 



## Расчет общей юнит-экономики

In [4]:
# Маркетинговый бюджет
ac_metric = sum(df['ltc'])

# Количество уникальных покупателей
b_metric = unique_buyers

# Общее количество юнитов
ua_metric = unique_uuids

# Количество сделок
t_metric = count_deals

# Средний чек
aov_metric = sum(df['aov'])/t_metric

# Переменные издержки
cogs_metric = sum(df['cogs'])/t_metric

# Стоимость привлечения юнита масштабирования
ltc_cpa_metric = ac_metric/ua_metric

# Конверсия
c1_metric = (b_metric/ua_metric)

# Среднее число сделок на одного клиента
apc_metric = t_metric/b_metric

# Средняя валовая прибыль на клиента
cltv_metric = (aov_metric - cogs_metric) * apc_metric

# Средняя валовая прибыль на юнит масштабирования
ltv_metric = cltv_metric * c1_metric

# Стоимость привлечения клиента
cac_metric = ac_metric/b_metric

# Маржинальная прибыль в юнит-экономике
cm_metric = ua_metric * (ltv_metric - ltc_cpa_metric)

# Выручка
revenue = b_metric * aov_metric * apc_metric

print(f'''AOV: {aov_metric:.2f},
UA: {ua_metric}
COGS: {cogs_metric}
LTC (CPA): {ltc_cpa_metric:.2f}
AC: {ac_metric}
C1: {c1_metric:.4f}
B: {b_metric}
T: {t_metric}
APC: {apc_metric:.2f}
CLTV: {cltv_metric:.2f}
LTV: {ltv_metric:.2f}
CAC: {cac_metric:.2f}
CM: {cm_metric:.2f}
Revenue: {revenue:.2f}
''')

AOV: 10.00,
UA: 41451
COGS: 1.3
LTC (CPA): 7.75
AC: 321365
C1: 0.0132
B: 547
T: 745
APC: 1.36
CLTV: 11.85
LTV: 0.16
CAC: 587.50
CM: -314883.50
Revenue: 7450.00



In [87]:
# df['month'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce')

## Метрики юнит-экономики по когортам

In [5]:
cohorts = df.groupby('cohort')

cohort_metrics = pd.DataFrame()

# Общее количество юнитов
cohort_metrics['UA'] = cohorts['uuid'].nunique()

# Количество уникальных покупателей
cohort_metrics['B'] = cohorts['buyer'].apply(lambda x: x[x != '-'].nunique())

# Количество сделок
cohort_metrics['T'] = cohorts['buyer'].apply(lambda x: (x != '-').sum())

# Маркетинговый бюджет
cohort_metrics['AC'] = cohorts['ltc'].sum()

# Средний чек
cohort_metrics['AOV'] = cohorts['aov'].sum() / cohort_metrics['T']

# Переменные издержки
cohort_metrics['COGS'] = cohorts['cogs'].sum() / cohort_metrics['T']

# Стоимость привлечения юнита масштабирования
cohort_metrics['LTC_CPA'] = cohort_metrics['AC'] / cohort_metrics['UA']

# Конверсия
cohort_metrics['C1'] = cohort_metrics['B'] / cohort_metrics['UA']

# Среднее число сделок на одного клиента
cohort_metrics['APC'] = cohort_metrics['T'] / cohort_metrics['B']

# Средняя валовая прибыль на клиента
cohort_metrics['CLTV'] = (cohort_metrics['AOV'] - cohort_metrics['COGS']) * cohort_metrics['APC']

# Средняя валовая прибыль на юнит масштабирования
cohort_metrics['LTV'] = cohort_metrics['CLTV'] * cohort_metrics['C1']

# Стоимость привлечения клиента
cohort_metrics['CAC'] = cohort_metrics['AC'] / cohort_metrics['B']

# Маржинальная прибыль в юнит-экономике
cohort_metrics['CM'] = cohort_metrics['UA'] * (cohort_metrics['LTV'] - cohort_metrics['LTC_CPA'])

# Выручка
cohort_metrics['Revenue'] = cohort_metrics['B'] * cohort_metrics['AOV'] * cohort_metrics['APC']

# totals = cohort_metrics.sum()

# cohort_metrics.loc['Total'] = totals

cohort_metrics.round(2)

,UA,B,T,AC,AOV,COGS,LTC_CPA,C1,APC,CLTV,LTV,CAC,CM,Revenue
cohort,,,,,,,,,,,,,,
2024-05,9810,197,328,113352,10.0,1.3,11.55,0.02,1.66,14.49,0.29,575.39,-110498.4,3280.0
2024-06,14547,201,268,118106,10.0,1.3,8.12,0.01,1.33,11.60,0.16,587.59,-115774.4,2680.0
2024-07,17094,149,149,89907,10.0,1.3,5.26,0.01,1.00,8.70,0.08,603.40,-88610.7,1490.0


In [106]:
# есть ли повторяющиеся uuid в разных когортах
multiple_cohorts = df.groupby('uuid')['cohort'].nunique()
print((multiple_cohorts > 1).sum()) 

0


## Метрики юнит-экономики по сегментам

In [6]:
df['buyer'] = df['buyer'].str.strip()  # убираем пробелы
# df = df[df['buyer'] != '-']  # убираем "дефисных"

# сколько раз каждый buyer встречается
buyer_counts = df[df['buyer'] != '-']['buyer'].value_counts()

# отбиарем тех, у кого покупок 2 и более
buyers_two_more = buyer_counts[buyer_counts >= 2].index

# Отфильтровываем датафрейм
df_two_more = df[df['buyer'].isin(buyers_two_more)]

df_two_more['buyer'].nunique()
df_two_more

,data,cohort,uuid,buyer,new buyer,aov,cogs,ltc,month,n_payment
162,5/1/2024,2024-05,628bd6ef-54a3-43cd-a486-a8237a23f976,e05134d8-2869-4f57-be00-d7fa5563d4de,True,10,1.3,8,2024-5,NaN
186,5/1/2024,2024-05,17a80c5f-8850-406d-81b8-9aa81185becb,30e59e49-7ff7-4c97-ad5e-22f1453994db,True,10,1.3,8,2024-5,NaN
188,5/1/2024,2024-05,974d924f-02d4-48e5-b6d5-990a020e05a6,6d87794b-606a-4206-adb4-cab1965906e0,True,10,1.3,0,2024-5,NaN
286,5/1/2024,2024-05,755b9471-de06-40ec-9cc6-d1f85429ebb8,5e44f82e-6f0b-4a30-a88f-7c61a300956c,True,10,1.3,8,2024-5,NaN
511,5/2/2024,2024-05,ad0ca654-cd75-422a-a04f-7161418e71af,b9b9f495-65d0-45e0-adbd-c4c37b570463,True,10,1.3,0,2024-5,NaN
...,...,...,...,...,...,...,...,...,...,...
41612,2024-07-29,2024-06,c106d493-0132-4228-92bf-b1aec7f088ec,5479dbb6-0684-42b7-8368-be35d4db2a3d,False,10,1.3,6,2024-7,NaN
41696,2024-07-29,2024-06,67bb63b0-4c90-4d96-a511-c270bcf186a0,da8915e3-0248-47b4-a9c2-aae8a5c86fe9,False,10,1.3,0,2024-7,NaN
41706,2024-07-29,2024-06,722c761b-6766-4de1-802f-251ff63af723,7f26a2f6-6405-4f7b-82ad-be93ab00c3d1,False,10,1.3,0,2024-7,NaN
41745,2024-07-29,2024-06,ebb3ab18-c637-4500-ba12-c38c05c2f103,21ba7265-2ee9-41e6-bada-1025274527e0,False,10,1.3,0,2024-7,NaN


In [34]:
cohort_metrics['AC']

cohort
2024-05    113352
2024-06    118106
2024-07     89907
Name: AC, dtype: int64

In [50]:
cohort_metrics_two_more['UA']

cohort
2024-05     9810
2024-06    14547
2024-07    17094
Name: UA, dtype: int64

In [51]:
cohort_metrics_two_more['LTC_CPA']

cohort
2024-05    11.554740
2024-06     8.118925
2024-07     5.259565
Name: LTC_CPA, dtype: float64

In [52]:
cohort_metrics_two_more['LTV']

cohort
2024-05    0.205749
2024-06    0.080140
2024-07    0.000000
Name: LTV, dtype: float64

In [53]:
cohort_metrics_two_more['UA']

cohort
2024-05     9810
2024-06    14547
2024-07    17094
Name: UA, dtype: int64

In [12]:
cohorts_two_more = df_two_more.groupby('cohort')

cohort_metrics_two_more = pd.DataFrame()

# Общее количество юнитов
cohort_metrics_two_more['UA'] = cohorts['uuid'].nunique()

# cohort_metrics_two_more['UA'] = cohorts_two_more['uuid'].nunique()

cohort_metrics_two_more['B'] = cohorts_two_more['buyer'].nunique()

cohort_metrics_two_more['T'] = cohorts_two_more.size()

cohort_metrics_two_more['AC'] = cohorts['ltc'].sum()

cohort_metrics_two_more['AOV'] = cohorts_two_more['aov'].sum() / cohort_metrics_two_more['T']

cohort_metrics_two_more['COGS'] = cohorts_two_more['cogs'].sum() / cohort_metrics_two_more['T']

cohort_metrics_two_more['LTC_CPA'] = cohort_metrics['AC'] / cohort_metrics_two_more['UA']

# cohort_metrics_two_more['LTC_CPA'] = cohort_metrics_two_more['AC'] / cohort_metrics_two_more['UA']

cohort_metrics_two_more['C1'] = cohort_metrics_two_more['B'] / cohort_metrics_two_more['UA']

cohort_metrics_two_more['APC'] = cohort_metrics_two_more['T'] / cohort_metrics_two_more['B']

cohort_metrics_two_more['CLTV'] = (cohort_metrics_two_more['AOV'] - cohort_metrics_two_more['COGS']) * cohort_metrics_two_more['APC']

cohort_metrics_two_more['LTV'] = cohort_metrics_two_more['CLTV'] * cohort_metrics_two_more['C1']

cohort_metrics_two_more['CAC'] = cohort_metrics_two_more['AC'] / cohort_metrics_two_more['B']

cohort_metrics_two_more['CM'] = cohort_metrics_two_more['UA'] * (cohort_metrics_two_more['LTV'] - cohort_metrics_two_more['LTC_CPA'])

# # Считаем LTC_CPA — всё делается автоматически, без хардкода
# cohort_metrics_two_more['LTC_CPA'] = cohort_metrics['AC'] / cohort_metrics_two_more['UA']

# # Считаем Contribution Margin (CM) по формуле
# cohort_metrics_two_more['CM'] = (cohort_metrics_two_more['UA'] * (cohort_metrics_two_more['LTV'] - cohort_metrics_two_more['LTC_CPA'])
# )

cohort_metrics_two_more['Revenue'] = cohort_metrics_two_more['B'] * cohort_metrics_two_more['AOV'] * cohort_metrics_two_more['APC']

#  Total (итоги по всем когортам)
# cohort_metrics_two_more.loc['Total'] = cohort_metrics_two_more.sum()


# cohort_metrics_two_more = cohort_metrics_two_more.fillna(0)
cohort_metrics_two_more.round(2)

,UA,B,T,AC,AOV,COGS,LTC_CPA,C1,APC,CLTV,LTV,CAC,CM,Revenue
cohort,,,,,,,,,,,,,,
2024-05,9810,101.0,232.0,113352,10.0,1.3,11.55,0.01,2.3,19.98,0.21,1122.30,-111333.6,2320.0
2024-06,14547,67.0,134.0,118106,10.0,1.3,8.12,0.00,2.0,17.40,0.08,1762.78,-116940.2,1340.0
2024-07,17094,NaN,NaN,89907,NaN,NaN,5.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
cohort_metrics_two_more['LTV']

cohort
2024-05    0.205749
2024-06    0.080140
2024-07    0.000000
Name: LTV, dtype: float64

In [41]:
cohort_metrics_two_more.dtypes

UA           int64
B          float64
T          float64
AC           int64
AOV        float64
COGS       float64
LTC_CPA    float64
C1         float64
APC        float64
CLTV       float64
LTV        float64
CAC        float64
CM         float64
Revenue    float64
dtype: object

In [42]:
print(cohort_metrics_two_more['LTV'].values)

[0.20574924 0.08014024 0.        ]


In [44]:
cohort_metrics_two_more['UA'] = 17094
cohort_metrics_two_more['LTV'] = 0.0

# Сначала LTC_CPA
cohort_metrics_two_more['LTC_CPA'] = cohort_metrics['AC'] / cohort_metrics_two_more['UA']

# Потом CM
cohort_metrics_two_more['CM'] = cohort_metrics_two_more['UA'] * (cohort_metrics_two_more['LTV'] - cohort_metrics_two_more['LTC_CPA'])

In [45]:
cohort_metrics_two_more.round(2)

,UA,B,T,AC,AOV,COGS,LTC_CPA,C1,APC,CLTV,LTV,CAC,CM,Revenue
cohort,,,,,,,,,,,,,,
2024-05,17094,101.0,232.0,113352,10.0,1.3,6.63,0.01,2.3,19.98,0.0,1122.30,-113352.0,2320.0
2024-06,17094,67.0,134.0,118106,10.0,1.3,6.91,0.00,2.0,17.40,0.0,1762.78,-118106.0,1340.0
2024-07,17094,0.0,0.0,89907,0.0,0.0,5.26,0.00,0.0,0.00,0.0,0.00,-89907.0,0.0


In [46]:
print(cohort_metrics_two_more[['LTC_CPA']].head())

          LTC_CPA
cohort           
2024-05  6.631099
2024-06  6.909208
2024-07  5.259565


In [47]:
print(cohort_metrics_two_more[['UA', 'LTC_CPA', 'CM']].head())

            UA   LTC_CPA        CM
cohort                            
2024-05  17094  6.631099 -113352.0
2024-06  17094  6.909208 -118106.0
2024-07  17094  5.259565  -89907.0


In [54]:
# Сначала LTV
cohort_metrics_two_more['LTV'] = cohort_metrics_two_more['CLTV'] * cohort_metrics_two_more['C1']

# Потом LTC_CPA
cohort_metrics_two_more['LTC_CPA'] = cohort_metrics['AC'] / cohort_metrics_two_more['UA']

# Потом CM
cohort_metrics_two_more['CM'] = (
    cohort_metrics_two_more['UA'] * (cohort_metrics_two_more['LTV'] - cohort_metrics_two_more['LTC_CPA'])
)

# Проверка
print(cohort_metrics_two_more[['UA', 'CLTV', 'C1', 'LTV', 'LTC_CPA', 'CM']].loc['2024-07'])

UA         17094.000000
CLTV           0.000000
C1             0.000000
LTV            0.000000
LTC_CPA        5.259565
CM        -89907.000000
Name: 2024-07, dtype: float64
